# Quickstart for Web Developers

> This page provides a fast introduction to FastHTML that covers a lot of topics that web developers find useful.

In [ ]:
#| default_exp core

::: {.callout-caution}
This document is a work in progress. 
:::

In [ ]:
#| hide
from nbdev.showdoc import *

## Installation

In [ ]:
pip install python-fasthtml

## A Minimal Application

A minimal FastHTML application looks something like this:

```{.python filename="main.py" code-line-numbers="true"}
from fasthtml.fastapp import * 

app, rt = fast_app()

@rt("/")
def get():
    return Titled("FastHTML", P("Let's do this!"))

run_uv()
```

What does that code do?

1. In line 1, we import a carefully-specified set of FastHTML functions and other Python objects into our global namespace, for rapid development. 

2. Next on line 3 we instantiate the FastHTML app with the `fast_app()` utility function. This provides a number of really useful defaults that we'll take advantage of later in the tutorial. 

3. Then on line 5 we use the `rt()` decorator to tell FastTML what URL should trigger our view function if visited with an HTTP GET. By HTTP GET we mean "go to that location in our browser"

4. Line 6 is where we define our function name of `get`. This also tells FastHTML which HTTP verb to use.

5. On line 7 we return several functions that describe all the HTML required to write a properly formed web page.

6. Finally, on line 10 the `run_uv()` utility configures and runs FastHTML using a library called `uvicorn`.

Run the code:

```bash
python main.py
```

The terminal will look like this:

```bash
INFO:     Uvicorn running on http://0.0.0.0:5001 (Press CTRL+C to quit)
INFO:     Started reloader process [58058] using WatchFiles
INFO:     Started server process [58060]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
```

Confirm FastHTML is running by opening your web browser to this linl: [127.0.0.1:5001](http://127.0.0.1:5001). You should see something like the image below:

![](quickstart-web-dev/quickstart-fasthtml.png)

## A Minimal Charting Application

The `Script` function allows you to include JavaScript. You can use Python to generate parts of your JS or JSON like this:

```python
import json
from fasthtml.common import * 

app, rt = fast_app(hdrs=(Script(src="https://cdn.plot.ly/plotly-2.32.0.min.js"),))

data = json.dumps({
    "data": [{"x": [1, 2, 3, 4],"type": "scatter"},
            {"x": [1, 2, 3, 4],"y": [16, 5, 11, 9],"type": "scatter"}],
    "title": "Plotly chart in FastHTML ",
    "description": "This is a demo dashboard",
    "type": "scatter"
})


@rt("/")
def get():
  return Titled("Chart Demo", Div(id="myDiv"),
    Script(f"var data = {json.loads(data)}; Plotly.newPlot('myDiv', data);"))

run_uv()
```

## Routing

FastHTML uses the Python community's friendly decorator pattern for specifying URLs:

```{.python filename="main.py" code-line-numbers="true"}
from fasthtml.fastapp import * 

app, rt = fast_app()

@rt("/")  # <1>
def get():
  return Titled("FastHTML", P("Let's do this!"))

@rt("/hello")  # <2>
def get():
  return Titled("Hello, world!")

run_uv()
```

1. The "/" URL on line 5 is the home of a project. This would be accessed at [127.0.0.1:5001](http://127.0.0.1:5001).
2. "/hello" URL on line 9 will be found by the project if the user visits [127.0.0.1:5001/hello](http://127.0.0.1:5001/hello).

::: {.callout-tip}
It looks like `get()` is being defined twice, but that's not the case. Each function decorated with `rt` is totally separate, and is injected into the router. We're not calling them in the module's  namespace (`locals()`). Rather, we're loading them into the routing mechanism using the `rt` decorator.
:::

You can do more! Read on to learn what we can do to make parts of the URL dynamic.

## Variables in URLs

You can add variable sections to a URL by marking them with `{variable_name}`. Your function then receives the `{variable_name}` as a keyword argument, but only if it is the correct type. Here's an example:

```{.python filename="main.py" code-line-numbers="true"}
from fasthtml.fastapp import * 

app, rt = fast_app()

@rt("/{name}/{age}")  # <3>
def get(name: str, age: int):  # <4>
  return Titled(f"Hello {name.title()}, age {age}")  # <5>

run_uv()
```

3. On line 3 we specify two variable names, `name` and `age`.
4. On line 4 we define two function arguments named identically to the variables. You will note that we specify the Python types to be passed.
5. On line 5, we use these functions in our project.


Try it out by going to this address: [127.0.0.1:5001/uma/5](http://127.0.0.1:5001/uma/5). You should get a page that says,

> "Hello Uma, age 5".


### What happens if we enter incorrect data?

The [127.0.0.1:5001/uma/5](http://127.0.0.1:5001/uma/5) URL works because `5` is an integer. If we enter something that is not, such as [127.0.0.1:5001/uma/five](http://127.0.0.1:5001/uma/five), then FastHTML will return an error instead of a web page.

::: {.callout-note}
### FastHTML URL routing supports more complex types

The two examples we provide here use Python's built-in `str` and `int` types, but you can use your own types, including more complex ones such as those defined by libraries like [attrs](https://pypi.org/project/attrs/), [pydantic](https://pypi.org/project/pydantic/), and even [sqlmodel](https://pypi.org/project/attrs/). 
:::

## HTTP Methods

FastHTML matches function names to HTTP methods. So far the URL routes we've defined have been for HTTP GET methods, the most common method for web pages.

Form submissions often are sent as HTTP POST. When dealing with more dynamic web page designs, also known as Single Page Apps (SPA for short), the need can arise for other methods such as HTTP PUT and HTTP DELETE. The way FastHTML handles this is by changing the function name.

```{.python filename="main.py" code-line-numbers="true"}
from fasthtml.fastapp import * 

app, rt = fast_app()

@rt("/")  
def get(): # <6>
  return Titled("HTTP GET", P("Handle GET"))

@rt("/")  
def post(): # <7>
  return Titled("HTTP POST", P("Handle POST"))

run_uv()
```

6. On line 6 because the `get()` function name is used, this will handle HTTP GETs going to the `/` URI.
7. On line 10 because the `post()` function name is used, this will handle HTTP POSTs going to the `/` URI.

## CSS Files and Inline Styles

Here we modify default headers to demonstrate how to use the [Sakura CSS microframework](https://github.com/oxalorg/sakura) instead of FastHTML's default of Pico CSS.

```{.python filename="main.py" code-line-numbers="true"}
from fasthtml.fastapp import * 

app, rt = fast_app(
    default_hdrs=False,  # <8>
    hdrs=(
        Link(rel='stylesheet', href='assets/normalize.min.css', type='text/css'),
        Link(rel='stylesheet', href='assets/sakura.css', type='text/css'),  # <9>
        Style("p {color: red;}")  # <10>
))

@app.get("/")
def home():
    return Titled("FastHTML",
        P("Let's do this!"),
    )

run_uv()
```

8. By setting `default_hdrs` to `False`, FastHTML will not include `pico.min.css`.
9. This will generate an HTML `<link>` tag for sourcing the css for Sakura.
10. If you want an inline styles, the `Style()` function will put the result into the HTML.

Check it out!

![](quickstart-web-dev/quickstart-sakura.png)

## Other Static Media Files

As you saw, `Script` and `Link` are specific to the most common static media use cases in web apps: including JavaScript and CSS.

FastHTML also provides a way to serve static media files. You can define a route that uses `FileResponse` to serve the file at the specified path. This is useful for serving images, videos, and other media files.

```{.python filename="main.py" code-line-numbers="true"}
@rt("/{fname:path}.{ext:static}")
async def get(fname:str, ext:str): 
    return FileResponse(f'public/{fname}.{ext}')
```

## Unwritten quickstart sections

1. Debug setting in fast_app
2. Rendering markdown
3. Code highlighting
4. Defining simple components of our own
5. Cookies
6. Sessions
7. Toast messages
8. BeforeWare and AfterWare
9. Websockets

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()